In [6]:
import numpy as np
from plotly import plotly
from plotly.grid_objs import Grid, Column
from plotly import graph_objs as go

Optimization Methods for Machine Learning

In [5]:
def f(x, y): return 0.4 * x ** 2 + x * y + 1.3 * y ** 2

def gradf(f , *xv, epsilon=1e-6):
    """
    Approximate the numerical gradient of f
    at around xv
    
    Parameters
    ----------
    f: function
        vectorized and derivable function
    xv: the evaluating points
    
    Returns
    -------
    np.array of shape (len(xv),);
    """
    nx = len(xv)
    xv_stack = np.array([xv for _ in range(nx)]).reshape(-1, nx)
    xv_stack_pos = (xv_stack + np.eye(nx) * epsilon).T
    xv_stack_neg = (xv_stack - np.eye(nx) * epsilon).T
    grad = (f(*xv_stack_pos) - f(*xv_stack_neg)) / (2 * epsilon)
    return grad

In [27]:
step = 0.5
xmin, xmax = -4, 4
ymin, ymax = -4, 4
xvals = np.arange(xmin, xmax, step)
yvals = np.arange(ymin, ymax, step)
X, Y = np.mgrid[xmin:xmax:step, ymin:ymax:step]
Z = f(X, Y)
Z.shape

(16, 16)

In [47]:
x0 = np.array([-3, -3])
alpha = 0.1
learning_history = [x0]
f1, f0 = f(*x0), 1e6
while abs(f1 / f0 - 1) > 1e-2:
    x0 = x0 - alpha * gradf(f, *x0)
    learning_history.append(x0)
    f0, f1 = f1, f(*x0)

learning_history = np.r_[learning_history]

In [113]:
x, y = learning_history.T
x = x[:50]
y = y[:50]
learning_xcols = [Column(x[:i+1], f"x{i}") for i, _ in enumerate(x)]
learning_ycols = [Column(y[:i+1], f"y{i}") for i, _ in enumerate(y)]


In [106]:
columns_mesh = [Column(xvals, "xvals"), Column(yvals, "yvals"), Column(Z, "name")]
columns_learn = [*learning_xcols, *learning_ycols]

In [90]:
grid_mesh = Grid(columns_mesh)
plotly.grid_ops.upload(grid_mesh, "mesh_grid", auto_open=False)

'https://plot.ly/~ger94/124/'

In [114]:
grid_learn = Grid(columns_learn)
plotly.grid_ops.upload(grid_learn, "learn01_grid", auto_open=False)

'https://plot.ly/~ger94/125/'

In [156]:
frames = [
    {"data": [
        {"xsrc": grid_learn.get_column_reference(f"x{i}"),
        "ysrc": grid_learn.get_column_reference(f"y{i}"),
        "mode": "markers"},
        
    ]
} for i, _ in enumerate(x)]

In [158]:
data = [
    go.Scatter(xsrc=grid_learn.get_column_reference("x0"),
               ysrc=grid_learn.get_column_reference("y0"),
               mode="markers"),
    go.Contour(xsrc=grid_mesh.get_column_reference("xvals"),
               ysrc=grid_mesh.get_column_reference("yvals"),
               zsrc=grid_mesh.get_column_reference("name"),
               colorscale="Viridis"),
]

layout = {
    "xaxis": {"range": [-4, 4], "autorange": False},
    "yaxis": {"range": [-4, 4], "autorange": False},
    "updatemenus": [{
        "buttons": [{
            "label": "Accion",
            "method": "animate",
            "args": [None,
                    {"frame": {"duration": 100, "redraw": False},
                     "fromcurrent":True,
                     "mode": "immediate"}]
        }],
        "pad": {"r":10, "t":87},
        "showactive": False,
        "type": "buttons"
    }]
}

figure = {
    "data": data,
    "layout": layout,
    "frames": frames
}

plotly.create_animations(figure, filename="learn01")

'https://plot.ly/~ger94/142/'

In [22]:
x0 = np.array([-3, -3])
alpha = 0.1
learning_history = [x0]
f1, f0 = f(*x0), 1e6
while abs(f1 / f0 - 1) > 1e-2:
    x0 = x0 - alpha * gradf(f, *x0)
    learning_history.append(x0)
    f0, f1 = f1, f(*x0)


learning_history = np.r_[learning_history]

In [23]:
xyrang = np.arange(-4, 4, step)
fig = go.FigureWidget(data=[
    go.Contour(z=Z, x=xyrang, y=xyrang, colorscale="Viridis", ncontours=30),
    go.Scatter(x=learning_history[:,1], y=learning_history[:,0], mode="lines+markers"),
])

fig

FigureWidget({
    'data': [{'colorscale': 'Viridis',
              'ncontours': 30,
              'type': 'co…